In [1]:
import pandas as pd
import skimage
import numpy as np
import sys
sys.path.append("../")
import torch
import os
import pathlib
os.chdir("/Users/ctromans/image-analysis/UNet_3D_C_elegans/data/")

from unet.utils.data_utils import generate_patches, save_patches

/Users/ctromans/miniconda3/envs/maddox-dbp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Make a patch dataset
# Basically, the ground truth is large 3D images (24, 1024, 1024) (z, y, x), so instead of generating patches on the fly
# before feeding to the model, this will allow for batching of different patches to be passed and hopefully 
# better model performance.

load_data = pd.read_csv("./data/data.csv")

patch_shape = (24, 200, 200)
stride_shape = (24, 100, 100)

save_df = {
    "image": [],
    "label": []
    }

for im, ma in zip(load_data.iloc[:, 0].values, load_data.iloc[:, 2].values):
    image = torch.from_numpy(skimage.io.imread(im).astype(np.float32))
    image_filename = os.path.basename(im)
    image_patches = generate_patches(image, patch_shape=patch_shape, stride_shape=stride_shape)
    image_save_paths = save_patches(image_patches, image_filename, "./patch_data/image")

    save_df["image"].extend(image_save_paths)

    mask = torch.from_numpy(skimage.io.imread(ma).astype(np.float32))
    mask_filename = os.path.basename(ma)
    mask_patches = generate_patches(mask, patch_shape=patch_shape, stride_shape=stride_shape)
    mask_save_paths = save_patches(mask_patches, mask_filename, "./patch_data/labels")

    save_df["label"].extend(mask_save_paths)

save_df = pd.DataFrame.from_dict(save_df, orient="index").transpose()
save_df.to_csv("./patch_data/load_data.csv", index=False)

In [14]:
# Make a new DF that excludes acquisition 1, which will be used for testing

df = pd.read_csv("../patch_data/load_data.csv")

df = df[~df["image"].str.contains("acqusition-1")]

df.to_csv("../patch_data/load_data_training.csv", index=False)

### Make multichannel patches with shape (c, z, spatial)

In [2]:
import pandas as pd
import os
os.chdir("/Users/ctromans/image-analysis/UNet_3D_C_elegans/data/")
import skimage

df = pd.read_csv("/Users/ctromans/image-analysis/UNet_3D_C_elegans/data/data.csv")

ch1_list, ch2_list = df.iloc[:, 0], df.iloc[:, 1]

save_dir = "./images_stacked_channels/"

os.makedirs(save_dir, exist_ok=True)

stacked_df = {
    "images": [],
    "masks": df.iloc[:, 2].values
}

img_list = []

for ch1, ch2 in zip(ch1_list, ch2_list):
    # Quick and dirty way to reconstruct filename
    filename = os.path.basename(ch1).rsplit("-", 1)[0] + "-stacked-channels." + os.path.basename(ch1).split(".")[-1]
    ch1_img, ch2_img = skimage.io.imread(ch1), skimage.io.imread(ch2)
    stacked_img = np.stack([ch1_img, ch2_img], axis=0)
    save_path = os.path.join(save_dir, filename)
    skimage.io.imsave(save_path, stacked_img)
    img_list.append(save_path)

stacked_df["images"] = img_list

stacked_df = pd.DataFrame.from_dict(stacked_df)
stacked_df.to_csv("/Users/ctromans/image-analysis/UNet_3D_C_elegans/data/data_stacked_channels.csv", index=False)


/var/folders/rw/691cc6h16f9cfxlh9g6bpq400000gq/T/ipykernel_13409/417812795.py:27: UserWarning: ./images_stacked_channels/mx85-nd-acqusition-0-stacked-channels.tif is a low contrast image
  skimage.io.imsave(save_path, stacked_img)
/var/folders/rw/691cc6h16f9cfxlh9g6bpq400000gq/T/ipykernel_13409/417812795.py:27: UserWarning: ./images_stacked_channels/mx85-nd-acqusition-1-stacked-channels.tif is a low contrast image
  skimage.io.imsave(save_path, stacked_img)
/var/folders/rw/691cc6h16f9cfxlh9g6bpq400000gq/T/ipykernel_13409/417812795.py:27: UserWarning: ./images_stacked_channels/mx85-nd-acqusition-8-stacked-channels.tif is a low contrast image
  skimage.io.imsave(save_path, stacked_img)


### Make multichannel patches

In [ ]:
%load_ext autoreload
%autoreload 2
from unet.utils.data_utils import generate_patches, save_patches


load_data = pd.read_csv("data_stacked_channels.csv")

patch_shape = (24, 200, 200)
stride_shape = (24, 100, 100)

save_df = {
    "images": [],
    "masks": []
    }

for im, ma in zip(load_data.iloc[:, 0].values, load_data.iloc[:, 1].values):
    image = torch.from_numpy(skimage.io.imread(im).astype(np.float32))
    image_filename = os.path.basename(im)
    image_patches = generate_patches(image, patch_shape=patch_shape, stride_shape=stride_shape, unfold_dims=[1, 2, 3])
    print(image_patches.shape)
    image_save_paths = save_patches(image_patches, image_filename, "./patch_data_stacked_channels/images", patch_dim=1)

    save_df["images"].extend(image_save_paths)

    mask = torch.from_numpy(skimage.io.imread(ma).astype(np.float32))
    mask_filename = os.path.basename(ma)
    # Mask can use default unfold dims
    mask_patches = generate_patches(mask, patch_shape=patch_shape, stride_shape=stride_shape)
    print(mask_patches.shape)
    mask_save_paths = save_patches(mask_patches, mask_filename, "./patch_data_stacked_channels/masks", patch_dim=0)

    save_df["masks"].extend(mask_save_paths)

save_df = pd.DataFrame.from_dict(save_df, orient="index").transpose()
save_df.to_csv("./patch_data_stacked_channels/load_data_stacked_channels.csv", index=False)

In [18]:
# Make a new DF that excludes acquisition 1, which will be used for testing

df = pd.read_csv("./patch_data_stacked_channels/load_data_stacked_channels.csv")

df = df[~df["images"].str.contains("acqusition-1")]

df.to_csv("./patch_data_stacked_channels/load_data_stacked_channels_training.csv", index=False)